### Reasoning (추론)

o3 및 o4-mini와 같은 추론 모델은 강화 학습을 통해 추론을 수행하도록 훈련된 LLM입니다. 추론 모델은 사용자에게 응답하기 전에 먼저 생각하여 , 사용자에게 응답하기 전에 긴 내부 사고 사슬을 생성합니다. 추론 모델은 복잡한 문제 해결, 코딩, 과학적 추론, 그리고 에이전트 워크플로우를 위한 다단계 계획 수립에 탁월합니다. 또한 , 경량 코딩 에이전트인 Codex CLI 에 가장 적합한 모델입니다.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
import json

client = OpenAI()

Model = "o4-mini"

In [3]:
prompt = """
문자열로 표현된 행렬을 '[1,2],[3,4],[5,6]' 형식으로 받아서 같은 형식으로 전치행렬을 출력하는 파이썬 스크립트를 작성하세요.
"""

response = client.responses.create(
    model="o4-mini",  
    reasoning={"effort": "medium"},  # 추론 수준 설정: 중간 수준의 정답 생성 요구
    input=[
        {
            "role": "user", 
            "content": prompt  # 사용자의 질문 또는 지시
        }
    ]
)

# 생성된 파이썬 스크립트 응답 출력
print(response.output_text)

다음은 입력 문자열 `"[1,2],[3,4],[5,6]"` 과 같은 형식으로 행렬을 받고, 전치행렬을 같은 형식으로 출력하는 예제입니다.

```python
import ast

def transpose_str_matrix(s: str) -> str:
    # "[1,2],[3,4],[5,6]" → "[ [1,2],[3,4],[5,6] ]" 로 감싸서 ast.literal_eval
    mat = ast.literal_eval('[' + s + ']')
    # 전치
    t = list(zip(*mat))
    # 다시 "[a,b],[c,d],..." 형태의 문자열로
    return ','.join('[' + ','.join(map(str, row)) + ']' for row in t)

if __name__ == '__main__':
    inp = input().strip()
    print(transpose_str_matrix(inp))
```

사용 예)  
입력:
[1,2],[3,4],[5,6]  
출력:
[1,3,5],[2,4,6]


In [4]:
response.output

[ResponseReasoningItem(id='rs_68620a46fa1481908a1c58d7cc6155000d1273bc68225041', summary=[], type='reasoning', encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_68620a4dfbc88190b3c64e7ce52bb53d0d1273bc68225041', content=[ResponseOutputText(annotations=[], text='다음은 입력 문자열 `"[1,2],[3,4],[5,6]"` 과 같은 형식으로 행렬을 받고, 전치행렬을 같은 형식으로 출력하는 예제입니다.\n\n```python\nimport ast\n\ndef transpose_str_matrix(s: str) -> str:\n    # "[1,2],[3,4],[5,6]" → "[ [1,2],[3,4],[5,6] ]" 로 감싸서 ast.literal_eval\n    mat = ast.literal_eval(\'[\' + s + \']\')\n    # 전치\n    t = list(zip(*mat))\n    # 다시 "[a,b],[c,d],..." 형태의 문자열로\n    return \',\'.join(\'[\' + \',\'.join(map(str, row)) + \']\' for row in t)\n\nif __name__ == \'__main__\':\n    inp = input().strip()\n    print(transpose_str_matrix(inp))\n```\n\n사용 예)  \n입력:\n[1,2],[3,4],[5,6]  \n출력:\n[1,3,5],[2,4,6]', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]

### Python 프로젝트 계획 및 생성

In [5]:
prompt = """
사용자 질문을 받아 답변과 매핑된 데이터베이스에서 검색하는 Python 앱을 만들고 싶습니다.
유사한 답변이 있으면 일치하는 답변을 검색합니다.
만약 일치하지 않으면 사용자에게 답변을 입력하도록 요청하고
질문/답변 쌍을 데이터베이스에 저장합니다. 필요한 디렉터리 구조를 계획한 후
각 파일을 전체적으로 반환합니다.
코드 전체가 아닌 시작과 끝에만 해설을 제공하세요.
한국어로 답하세요.
"""

response = client.responses.create(
    model="o4-mini",
    input=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)

print(response.output_text)

아래와 같은 디렉터리 구조를 제안합니다.

project_root/  
├─ app.py  
├─ requirements.txt  
├─ db/  
│  ├─ __init__.py  
│  ├─ database.py  
│  └─ models.py  
└─ services/  
   ├─ __init__.py  
   ├─ retriever.py  
   └─ writer.py  

각 파일의 전체 내용은 다음과 같습니다.

파일: db/__init__.py  
(빈 파일)

파일: db/database.py
```
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///db/qa.db', connect_args={'check_same_thread': False})
SessionLocal = sessionmaker(bind=engine)
Base = declarative_base()

def init_db():
    Base.metadata.create_all(bind=engine)
```

파일: db/models.py
```
from sqlalchemy import Column, Integer, String
from .database import Base

class QA(Base):
    __tablename__ = 'qa'
    id = Column(Integer, primary_key=True, index=True)
    question = Column(String, unique=True, index=True)
    answer = Column(String)
```

파일: services/__init__.py  
(빈 파일)

파일: services/retriever.py
```
import nump

In [6]:
response.output

[ResponseReasoningItem(id='rs_68620a6665b481938518cb4bd7285aae0657dedb649276a1', summary=[], type='reasoning', encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_68620a713a14819381dccea1e3aa14bb0657dedb649276a1', content=[ResponseOutputText(annotations=[], text='아래와 같은 디렉터리 구조를 제안합니다.\n\nproject_root/  \n├─ app.py  \n├─ requirements.txt  \n├─ db/  \n│  ├─ __init__.py  \n│  ├─ database.py  \n│  └─ models.py  \n└─ services/  \n   ├─ __init__.py  \n   ├─ retriever.py  \n   └─ writer.py  \n\n각 파일의 전체 내용은 다음과 같습니다.\n\n파일: db/__init__.py  \n(빈 파일)\n\n파일: db/database.py\n```\nfrom sqlalchemy import create_engine\nfrom sqlalchemy.ext.declarative import declarative_base\nfrom sqlalchemy.orm import sessionmaker\n\nengine = create_engine(\'sqlite:///db/qa.db\', connect_args={\'check_same_thread\': False})\nSessionLocal = sessionmaker(bind=engine)\nBase = declarative_base()\n\ndef init_db():\n    Base.metadata.create_all(bind=engine)\n```\n\n파일: db/models.py\n```\nfrom sqlalchemy

## 추론 기반 코드 자동 생성 실습

**"숫자로 이루어진 문자열이 주어질 때, 해당 숫자의 평균과 중앙값을 구하는 Python 함수를 작성하라."**
예를 들어 `"1, 5, 3, 8, 2"`라는 문자열이 입력되면 평균은 `3.8`, 중앙값은 `3`입니다.


* `o4-mini` 추론 모델을 사용하여 위 문제를 해결하는 **Python 코드**를 생성하세요.
* `reasoning={"effort": "medium"}` 또는 `reasoning={"effort": "high"}` 옵션을 사용하여 **모델의 내부 추론 과정**을 유도하세요.
* 코드 실행 결과가 **정확한지 직접 검증**해보세요.

### 구현 조건

* 문자열을 숫자 리스트로 변환하는 전처리 로직 포함
* 평균(average)과 중앙값(median)을 계산하는 함수 구성
* 코드 내에 주석 또는 단계별 설명이 포함되도록 모델에게 유도
* 출력값 확인을 위한 예제 입력 포함 (`"1, 5, 3, 8, 2"`)

* reasoning effort를 `low`, `medium`, `high`로 바꿔가며 결과물 비교
* `json.loads()`로 실행 가능한 함수 형태로 모델이 응답하도록 유도해보기